In [ ]:
%pip install -r requirements.txt

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import random
import time

In [ ]:
with open("session1.txt", "r") as f:
    for line in f:
        if "sessionid=" in line:
            SESSIONID = line.strip().split("=", 1)[1]
        elif "link_to_folder=" in line:
            LINK_TO_SAVED_FOLDER = line.strip().split("=", 1)[1]

if not SESSIONID: 
    print("sessionid is not found in the session1.txt")
    
if not LINK_TO_SAVED_FOLDER: 
    print("link_to_folder is not found in the session1.txt")


In [ ]:
# Start browser
driver = webdriver.Chrome()
driver.get("https://www.instagram.com/")

# Set session cookie
driver.add_cookie({
    "name": "sessionid",
    "value": SESSIONID,
    "domain": ".instagram.com",
    "path": "/"
})

# Navigate to saved posts
driver.get(LINK_TO_SAVED_FOLDER)

time.sleep(5)  # wait for possible redirect

if "/accounts/login/" in driver.current_url:
    print("Login failed: Invalid or expired session_id.")
    driver.quit()
else:
    print("Logged in successfully.")
    MAX_SCROLL_TRIES = 5  # How many times to retry scrolling when no new posts are loaded

    post_links = set()
    scroll_tries = 0

    last_height = driver.execute_script("return document.body.scrollHeight")

    while scroll_tries < MAX_SCROLL_TRIES:
        # collect current links
        posts = driver.find_elements(By.TAG_NAME, "a")
        for elem in posts: 
            href = elem.get_attribute("href")
            if href and "/p/" in href:
                post_links.add(href)
        
        #scroll to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        #add random stops to let content load and to avoid ban
        time.sleep(random.randint(3,7))

        # Check if new content was loaded
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            scroll_tries += 1
        else:
            scroll_tries = 0  # Reset if content was loaded

        last_height = new_height

    print(f"{len(post_links)} posts found")
    #save in a file
    with open("insta_links.txt", "w") as f:
        for link in post_links:
            f.write(link + "\n")

    driver.quit()